In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager    # 매 번 크롬 드라이버를 설치할 필요없이 자동으로
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert
from tqdm import tqdm_notebook
from urllib.request import urlopen
import time
import requests
from datetime import datetime, timedelta

## 1) Total_audience 전처리

In [3]:
audience = pd.read_csv("Total_audience.csv", index_col=0, header=[0,1])
audience.rename(columns={"Unnamed: 0_level_1":""}, inplace=True)

In [4]:
audience.head()
audience.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7268 entries, 0 to 7267
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   (연도, )      7268 non-null   object 
 1   (한국, 개봉편수)  7268 non-null   int64  
 2   (한국, 상영편수)  7268 non-null   int64  
 3   (한국, 매출액)   7268 non-null   int64  
 4   (한국, 관객수)   7268 non-null   int64  
 5   (한국, 점유율)   7268 non-null   float64
 6   (외국, 개봉편수)  7268 non-null   int64  
 7   (외국, 상영편수)  7268 non-null   int64  
 8   (외국, 매출액)   7268 non-null   int64  
 9   (외국, 관객수)   7268 non-null   int64  
 10  (외국, 점유율)   7268 non-null   float64
 11  (전체, 개봉편수)  7268 non-null   int64  
 12  (전체, 상영편수)  7268 non-null   int64  
 13  (전체, 매출액)   7268 non-null   int64  
 14  (전체, 관객수)   7268 non-null   int64  
dtypes: float64(2), int64(12), object(1)
memory usage: 908.5+ KB


## 1-1) 필요한 열과 필요한 행만 가져오기

### (1) 필요한 열만 가져오기

In [5]:
audience = audience.iloc[:,[0,11,12,13,14]]      # 연도, 전체 개봉편수, 전체 상영편수, 전체 매출액, 전체 관객수

In [6]:
audience.head()

연도   전체                        
                       개봉편수 상영편수        매출액     관객수
0  2004-01-01 00:00:00    1    7  727005000  113837
1  2004-01-02 00:00:00    0    7  642276500  104357
2  2004-01-03 00:00:00    0    7  796519500  125234
3  2004-01-04 00:00:00    0    7  763271000  120203
4  2004-01-05 00:00:00    0    7  409269500   64180

### (2) 필요 없는 행 제거해주기

- 각 월 마다 통계치를 합계해 준 행들이 있음

In [7]:
audience.loc[audience.loc[:,"연도"]=="합계",:]

연도   전체                              
         개봉편수  상영편수           매출액       관객수
31    합계   13   295   19429676000   3076067
61    합계   28   420   19538678000   3113385
93    합계   21   541   11777867500   1845368
124   합계   35   573   20828686000   3307663
156   합계   14   519   24678301700   3817166
...   ..  ...   ...           ...       ...
7167  합계  132  3157  157645792126  14171771
7199  합계  124  2661  124073209373  11250357
7228  합계  104  2571   69050497975   6421295
7260  합계  145  2738   80022730615   7476215
7267  합계   16   532   11863437549   1172168

[232 rows x 5 columns]

In [8]:
audience.drop(audience[audience.loc[:,"연도"]=="합계"].index, inplace = True)    # 7268 행 -> 7036 행

In [9]:
audience.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7036 entries, 0 to 7266
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   (연도, )      7036 non-null   object
 1   (전체, 개봉편수)  7036 non-null   int64 
 2   (전체, 상영편수)  7036 non-null   int64 
 3   (전체, 매출액)   7036 non-null   int64 
 4   (전체, 관객수)   7036 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 329.8+ KB


## 1-2) 평일과 주말 데이터를 나누기 위해 "요일" 파생변수 추가

In [29]:
audience.info

<bound method DataFrame.info of                        연도   전체                            요일
                          개봉편수 상영편수         매출액     관객수     
0     2004-01-01 00:00:00    1    7   727005000  113837  Thu
1     2004-01-02 00:00:00    0    7   642276500  104357  Fri
2     2004-01-03 00:00:00    0    7   796519500  125234  Sat
3     2004-01-04 00:00:00    0    7   763271000  120203  Sun
4     2004-01-05 00:00:00    0    7   409269500   64180  Mon
...                   ...  ...  ...         ...     ...  ...
7262  2023-04-02 00:00:00    0   80  3423512979  336191  Sun
7263  2023-04-03 00:00:00    0   87   802358725   81632  Mon
7264  2023-04-04 00:00:00    2   91   815279367   82221  Tue
7265  2023-04-05 00:00:00    6   94  1347838240  134732  Wed
7266  2023-04-06 00:00:00    7  100  1058649412  107135  Thu

[7036 rows x 6 columns]>

In [11]:
audience['요일'] = pd.to_datetime(audience["연도"],format="%Y-%m-%d").dt.strftime('%a') 

In [13]:
audience.head(3)
audience.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7036 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   (연도, )      7036 non-null   object
 1   (전체, 개봉편수)  7036 non-null   int64 
 2   (전체, 상영편수)  7036 non-null   int64 
 3   (전체, 매출액)   7036 non-null   int64 
 4   (전체, 관객수)   7036 non-null   int64 
 5   (요일, )      7036 non-null   object
dtypes: int64(4), object(2)
memory usage: 384.8+ KB


## 1-3) 이중 컬럼을 하나의 컬럼으로 합쳐주기

In [83]:
print(audience.columns.get_level_values(0))
len(audience.columns.get_level_values(0))

Index(['연도', '전체', '전체', '전체', '전체', '요일'], dtype='object')


6

In [84]:
print(audience.columns.get_level_values(1))
len(audience.columns.get_level_values(1))

Index(['', '개봉편수', '상영편수', '매출액', '관객수', ''], dtype='object')


6

In [85]:
audience1 = audience.copy()    # 혹시 모를 경우를 대비해 사본으로 만들기

In [86]:
audience1.columns = [audience.columns.get_level_values(0)[i] + audience.columns.get_level_values(1)[i] for
                    i in range(0,len(audience.columns.get_level_values(0)))]

In [138]:
audience1["연도"] = pd.to_datetime(audience1["연도"], format="%Y-%m-%d")

In [87]:
audience1.head(3)

연도  전체개봉편수  전체상영편수      전체매출액   전체관객수   요일
0 2004-01-01       1       7  727005000  113837  Thu
1 2004-01-02       0       7  642276500  104357  Fri
2 2004-01-03       0       7  796519500  125234  Sat

In [141]:
audience1.to_csv("audience1_preprocess1.csv", encoding = "utf-8", index= True, header=True)     # 이중 컬럼 합친 것.

## read_csv 하면 Dtype 이 문자열로 풀려서 실행해줘야 함

In [71]:
audience1 = pd.read_csv("audience1_preprocess1.csv", encoding = "utf-8", index_col=0)
audience1.head(3)

연도  전체개봉편수  전체상영편수      전체매출액   전체관객수   요일
0  2004-01-01       1       7  727005000  113837  Thu
1  2004-01-02       0       7  642276500  104357  Fri
2  2004-01-03       0       7  796519500  125234  Sat

In [72]:
audience1["연도"] = pd.to_datetime(audience1["연도"], format="%Y-%m-%d")

In [74]:
audience1.info()
audience1.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7036 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   연도      7036 non-null   datetime64[ns]
 1   전체개봉편수  7036 non-null   int64         
 2   전체상영편수  7036 non-null   int64         
 3   전체매출액   7036 non-null   int64         
 4   전체관객수   7036 non-null   int64         
 5   요일      7036 non-null   object        
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 384.8+ KB


연도  전체개봉편수  전체상영편수      전체매출액   전체관객수   요일
0 2004-01-01       1       7  727005000  113837  Thu
1 2004-01-02       0       7  642276500  104357  Fri
2 2004-01-03       0       7  796519500  125234  Sat

## 2. Gni 전처리

In [30]:
Gni = pd.read_csv("C:\\Users\\user\\Downloads\\주요지표_연간지표__20230407142721.csv",
                 encoding = "euc-kr", index_col=0)

In [32]:
Gni.T

계정항목별  국내총생산(명목, 원화표시) (십억원)  국내총생산(명목, 달러표시) (억달러)  국민총소득(명목, 원화표시) (십억원)  \
2003                837365.0                 7025.5               834443.2   
2004                908439.2                 7936.3               906864.7   
2005                957447.8                 9347.2               950685.4   
2006               1005601.5                10524.2              1002664.7   
2007               1089660.2                11726.9              1086897.3   
2008               1154216.5                10468.2              1154509.7   
2009               1205347.7                 9443.3              1203479.8   
2010               1322611.2                11438.7              1324586.9   
2011               1388937.2                12534.3              1397534.8   
2012               1440111.4                12779.6              1455170.3   
2013               1500819.1                13705.6              1510384.9   
2014               1562928.9                14839.5              1570493.3   
2015               1658020.4                14653.4              1663206.6   
2016               1740779.6                15000.3              1747143.5   
2017               1835698.2                16233.1              1843180.9   
2018               1898192.6                17251.6              1905837.5   
2019               1924498.1                16510.1              1941107.9   
2020               1940726.2                16446.1              1957669.9   
2021               2071658.0                18102.3              2094721.3   
2022               2150575.8                16643.3              2178875.7   

계정항목별  국민총소득(명목, 달러표시) (억달러)  요소비용국민소득(명목) (십억원)  국민처분가능소득(명목) (십억원)  \
2003                  7001.0            602692.2            689409.2   
2004                  7922.5            662698.2            750340.6   
2005                  9281.2            691776.2            784904.9   
2006                 10493.5            728236.3            827011.2   
2007                 11697.1            792507.9            901343.3   
2008                 10470.9            828123.1            946644.4   
2009                  9428.7            859286.3            976066.0   
2010                 11455.8            953725.5           1080250.2   
2011                 12611.9           1002826.4           1135618.1   
2012                 12913.3           1041626.9           1177260.3   
2013                 13793.0           1086725.1           1224415.2   
2014                 14911.4           1128904.1           1271899.3   
2015                 14699.3           1196864.4           1349292.2   
2016                 15055.1           1256956.7           1419199.9   
2017                 16299.2           1326542.8           1497065.8   
2018                 17321.1           1364946.0           1539485.3   
2019                 16652.6           1374935.0           1553455.0   
2020                 16589.7           1365736.5           1554750.8   
2021                 18303.8           1450859.9           1658266.8   
2022                 16862.3                 0.0                 0.0   

계정항목별  국민총처분가능소득(명목) (십억원)  가계총처분가능소득(명목) (십억원)  1인당 국내총생산(명목, 원화표시) (만원)  \
2003              831292.4             494340.2                    1748.4   
2004              904224.4             532359.3                    1889.3   
2005              948273.9             556248.7                    1987.0   
2006              999356.0             581833.5                    2076.0   
2007             1084398.3             617108.9                    2238.2   
2008             1154672.2             658676.7                    2352.9   
2009             1202422.0             681634.6                    2444.5   
2010             1319387.5             722576.0                    2669.0   
2011             1393483.2             762753.3                    2781.4   
2012             1450611.4             793887.9                    2868.8   
2

In [33]:
Gni.to_csv("Gni.csv", encoding="utf-8", index=True, header=True)

## 평일,주말 가격표 만들기

In [2]:
from datetime import datetime, timedelta

start_date = datetime(2004, 1, 1)
end_date = datetime(2023, 4, 6)
date = pd.date_range(start_date, end_date)

price = pd.DataFrame(date, columns=['date'])
price['date'] = price['date'].dt.strftime('%Y-%m-%d')

In [3]:
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7036 entries, 0 to 7035
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    7036 non-null   object
dtypes: object(1)
memory usage: 55.1+ KB


In [4]:
price['date'] = price["date"].astype(date)

price.head(3)
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7036 entries, 0 to 7035
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    7036 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 55.1 KB


In [5]:
price['요일'] = pd.to_datetime(price["date"],format="%Y-%m-%d %H:%M:%S").dt.strftime('%a')

In [6]:
price.head(3)

date   요일
0 2004-01-01  Thu
1 2004-01-02  Fri
2 2004-01-03  Sat

## 가격 설정해주기 by 프라임 타임

In [7]:
price['가격'] = 0       # 일단 0으로 채워넣음.

In [8]:
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7036 entries, 0 to 7035
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    7036 non-null   datetime64[ns]
 1   요일      7036 non-null   object        
 2   가격      7036 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 165.0+ KB


### 프라임 타임 ( 피크 타임)

----

- 월요일 ~ 목요일은 평일 요금 / 금요일 ~ 일요일은 주말 요금을 받는다.
------
- 2004년 1월 1일  ~ 7000원
- 2004년 7월 1일,주중 7000원, 주말 8000원
---
- 2009년 7월 3일부터 일반 평일 요금은 8천원, 주말 요금은 9천원 
---
- 2013년 2월 14일부터 평일 요금 9천 / 주말 요금 1만원
- 2016년 3월 3일부터 평일 1만원 / 주말 11000원
- 2018년 4월 11일 평일:11000 / 주말:12000  
- 2020년 10월 26일 좌석 차등제 폐지  평일 : 12000 / 주말 : 13000
- 2021년 4월 2일 평일: 13000/ 주말: 14000

### 행 확인

In [10]:
price.loc[price["date"] == "2004-07-01 00:00:00"]    # 182행
price.loc[price["date"] == "2009-07-02 00:00:00"]    # 183:2009행       # 공휴일 8000원
price.loc[price["date"] == "2013-02-13 00:00:00"]    # 2010:3331행      # 공휴일 9000원
price.loc[price["date"] == "2016-03-02 00:00:00"]    # 3332:4444행      # 공휴일 10000원
price.loc[price["date"] == "2018-04-10 00:00:00"]    # 4445:5213행      # 공휴일: 11000원
price.loc[price["date"] == "2020-10-25 00:00:00"]    # 5214:6142행      # 공휴일 : 12000원
price.loc[price["date"] == "2021-04-01 00:00:00"]    # 6300행           # 공휴일 : 13000원

date   요일  가격
6300 2021-04-01  Thu   0

#### 2004년 7월 1일 까지 주중, 주말 7000원

In [11]:
price.loc[0:182, '가격'] = 7000

#### 2004년 7월 2일 부터 2009 년 7월 2일 까지 주중 7000원, 주말 8000원

In [12]:
price.loc[183:2009, '가격'] = price.loc[183:2009, '요일'].apply(lambda x: 8000 if x in ['Fri', 'Sat', 'Sun'] else 7000)

#### 2009년 7월 3일 부터 2013 년 2월 13일 까지 주중 8000원, 주말 9000원

In [13]:
price.loc[2010:3331, '가격'] = price.loc[2010:3331, '요일'].apply(lambda x: 9000 if x in ['Fri', 'Sat', 'Sun'] else 8000)

#### 2013년 2월 14일부터 2016년 3월 2일까지 평일 요금 9천 / 주말 요금 1만원

In [14]:
price.loc[3332:4444, '가격'] = price.loc[3332:4444, '요일'].apply(lambda x: 10000 if x in ['Fri', 'Sat', 'Sun'] else 9000)

#### 2016년 3월 3일부터 2018년 4월 10일까지 평일 요금 1만원  / 주말 요금 11000원

In [15]:
price.loc[4445:5213, '가격'] = price.loc[4445:5213, '요일'].apply(lambda x: 11000 if x in ['Fri', 'Sat', 'Sun'] else 10000)

#### 2018년 4월 11일부터 2020년 10월 25일까지 평일 요금 11000원 주말:12000원

In [16]:
price.loc[5214:6142, '가격'] = price.loc[5214:6142, '요일'].apply(lambda x: 12000 if x in ['Fri', 'Sat', 'Sun'] else 11000)

#### 2020년 10월 26일부터 2021 년 4월 1일까지 좌석 차등제 폐지 평일 12000원  주말 : 13000원

In [17]:
price.loc[6143:6300, '가격'] = price.loc[6143:6300, '요일'].apply(lambda x: 13000 if x in ['Fri', 'Sat', 'Sun'] else 12000)

#### 2021년 4월 2일부터 현재까지 평일: 13000원 주말: 14000원

In [18]:
price.loc[6301:, '가격'] = price.loc[6301:, '요일'].apply(lambda x: 14000 if x in ['Fri', 'Sat', 'Sun'] else 13000)

In [20]:
# 중간 확인
price.loc[200:300].head(50)

date   요일    가격
200 2004-07-19  Mon  7000
201 2004-07-20  Tue  7000
202 2004-07-21  Wed  7000
203 2004-07-22  Thu  7000
204 2004-07-23  Fri  8000
205 2004-07-24  Sat  8000
206 2004-07-25  Sun  8000
207 2004-07-26  Mon  7000
208 2004-07-27  Tue  7000
209 2004-07-28  Wed  7000
210 2004-07-29  Thu  7000
211 2004-07-30  Fri  8000
212 2004-07-31  Sat  8000
213 2004-08-01  Sun  8000
214 2004-08-02  Mon  7000
215 2004-08-03  Tue  7000
216 2004-08-04  Wed  7000
217 2004-08-05  Thu  7000
218 2004-08-06  Fri  8000
219 2004-08-07  Sat  8000
220 2004-08-08  Sun  8000
221 2004-08-09  Mon  7000
222 2004-08-10  Tue  7000
223 2004-08-11  Wed  7000
224 2004-08-12  Thu  7000
225 2004-08-13  Fri  8000
226 2004-08-14  Sat  8000
227 2004-08-15  Sun  8000
228 2004-08-16  Mon  7000
229 2004-08-17  Tue  7000
230 2004-08-18  Wed  7000
231 2004-08-19  Thu  7000
232 2004-08-20  Fri  8000
233 2004-08-21  Sat  8000
234 2004-08-22  Sun  8000
235 2004-08-23  Mon  7000
236 2004-08-24  Tue  7000
237 2004-08-25  Wed  7000
238 2004-08-26  Thu  7000
239 2004-08-27  Fri  8000
240 2004-08-28  Sat  8000
241 2004-08-29  Sun  8000
242 2004-08-30  Mon  7000
243 2004-08-31  Tue  7000
244 2004-09-01  Wed  7000
245 2004-09-02  Thu  7000
246 2004-09-03  Fri  8000
247 2004-09-04  Sat  8000
248 2004-09-05  Sun  8000
249 2004-09-06  Mon  7000

## 공휴일도 주말 요금을 받기 때문에, 요금 바꿔주기

In [21]:
import requests
import datetime
import pandas as pd
from bs4 import BeautifulSoup
import urllib.parse as urlparse


def get_request_query(url, operation, params, serviceKey):
    params = urlparse.urlencode(params)
    request_query = url + '/' + operation + '?' + params + '&' + 'serviceKey' + '=' + serviceKey
    return request_query

mykey = "ApRZ95DtQHw7scnx1D%2FHIOoxouuIvvx%2BSm86AbFRaJS4WqS7EaGQASlMbd0y7aLAuuuFla867EYaGau0blJFIA%3D%3D"
url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService'
operation = 'getRestDeInfo'

df = pd.DataFrame()

for yr in range(2002, 2024):
    for month in range(1,13):

        if month < 10:
            month = '0' + str(month)
        else:
            month = str(month)
        params = {'solYear':yr, 'solMonth':month}

        request_query = get_request_query(url, operation, params, mykey)

        res = requests.get(request_query)
        soup = BeautifulSoup(res.text, 'lxml')
        items = soup.find_all('item')

        for item in items:
            day = item.locdate.get_text()
            name = item.datename.get_text()
            data = {'holiday':name , 'date':day}
            df = df.append(data, ignore_index=True)
    
display(df.head())
display(df.tail())

C:\Users\user\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_7580\3553883377.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_7580\3553883377.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Use

C:\Users\user\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_7580\3553883377.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\user\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this

C:\Users\user\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_7580\3553883377.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\user\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this

C:\Users\user\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_7580\3553883377.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\user\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this

C:\Users\user\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_7580\3553883377.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\user\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this

...

C:\Users\user\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_7580\3553883377.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\user\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this

C:\Users\user\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_7580\3553883377.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\user\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this

C:\Users\user\AppData\Local\Temp\ipykernel_7580\3553883377.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Users\user\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_7580\3553883377.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
C:\Use

holiday      date
0      신정  20040101
1      설날  20040121
2      설날  20040122
3      설날  20040123
4     삼일절  20040301

holiday      date
319      추석  20230929
320      추석  20230930
321     개천절  20231003
322     한글날  20231009
323   기독탄신일  20231225

In [22]:
df1 = df.copy()   # 혹시 모를 상황을 대비해 사본 마련해두기

In [30]:
df.head()

holiday      date
0      신정  20040101
1      설날  20040121
2      설날  20040122
3      설날  20040123
4     삼일절  20040301

In [31]:
from datetime import datetime

idx = 0
for d in df["date"]:
    df["date"][idx] = datetime.strptime(str(df["date"][idx]), "%Y%m%d")
    idx += 1

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   holiday  324 non-null    object
 1   date     324 non-null    object
dtypes: object(2)
memory usage: 5.2+ KB


In [33]:
df["date"] = df["date"].apply(lambda x: datetime.strftime(x, "%Y-%m-%d"))

In [34]:
df["date"] = df["date"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))  # datetime 형태로 맞춰주기

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   holiday  324 non-null    object        
 1   date     324 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 5.2+ KB


In [36]:
df.head()

holiday       date
0      신정 2004-01-01
1      설날 2004-01-21
2      설날 2004-01-22
3      설날 2004-01-23
4     삼일절 2004-03-01

In [37]:
df.value_counts("date")   # 공휴일에 중복이 있는지 확인.  # 중복이 있음.

date
2017-10-03    2
2009-10-03    2
2006-05-05    2
2017-10-05    1
2018-02-17    1
             ..
2011-05-05    1
2011-03-01    1
2011-02-04    1
2011-02-03    1
2023-12-25    1
Length: 321, dtype: int64

### 중복이 있는 행 찾기

In [38]:
df.loc[df['date']=='2006-05-05',:]

holiday       date
37    어린이날 2006-05-05
38   석가탄신일 2006-05-05

In [39]:
df.drop(38, inplace=True)   # 삭제

In [40]:
df.loc[df['date']=='2009-10-03',:]

holiday       date
86      추석 2009-10-03
87     개천절 2009-10-03

In [41]:
df.drop(87, inplace=True)   # 삭제

In [42]:
df.loc[df['date']=='2017-10-03',:]

holiday       date
212     개천절 2017-10-03
213      추석 2017-10-03

In [43]:
df.drop(213, inplace=True)  # 삭제

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 321 entries, 0 to 323
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   holiday  321 non-null    object        
 1   date     321 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 7.5+ KB


## 중간 체크 리스트 만들기

In [45]:
df.to_csv("holiday.csv", encoding = "utf-8", index= True, header=True)

In [46]:
price.head()
price.rename(columns={"연도":"date"}, inplace=True)     # 머지를 위해 on 조건으로 쓰일 컬럼 맞춰주기

In [51]:
price.info()
price.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7036 entries, 0 to 7035
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    7036 non-null   datetime64[ns]
 1   요일      7036 non-null   object        
 2   가격      7036 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 165.0+ KB


date   요일    가격
0 2004-01-01  Thu  7000
1 2004-01-02  Fri  7000
2 2004-01-03  Sat  7000

In [52]:
df.info()
df.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 321 entries, 0 to 323
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   holiday  321 non-null    object        
 1   date     321 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 7.5+ KB


holiday       date
0      신정 2004-01-01
1      설날 2004-01-21
2      설날 2004-01-22

## 가격 데이터 프레임과 공휴일 데이터 프레임 합치기

In [53]:
all_date = pd.merge(price, df, how='left')   # 머지 해주기

In [55]:
all_date.info()
all_date.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7036 entries, 0 to 7035
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     7036 non-null   datetime64[ns]
 1   요일       7036 non-null   object        
 2   가격       7036 non-null   int64         
 3   holiday  311 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 274.8+ KB


date   요일    가격 holiday
0 2004-01-01  Thu  7000      신정
1 2004-01-02  Fri  7000     NaN
2 2004-01-03  Sat  7000     NaN

In [56]:
## 공휴일 결측치를 0으로 채우기 ##
all_date["holiday"] = all_date["holiday"].fillna(0)

In [57]:
all_date.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7036 entries, 0 to 7035
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     7036 non-null   datetime64[ns]
 1   요일       7036 non-null   object        
 2   가격       7036 non-null   int64         
 3   holiday  7036 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 274.8+ KB


In [58]:
## 공휴일인 날을 1로 바꾸기 ##
for i in range(len(all_date["holiday"])):
    if all_date["holiday"][i] != 0:
        all_date["holiday"][i] = "1"


C:\Users\user\AppData\Local\Temp\ipykernel_7580\1658010662.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_date["holiday"][i] = "1"


In [59]:
all_date["holiday"] = pd.to_numeric(all_date["holiday"])

In [61]:
all_date.head(3)    # 공휴일이면 1, 공휴일이 아니면 0

date   요일    가격  holiday
0 2004-01-01  Thu  7000        1
1 2004-01-02  Fri  7000        0
2 2004-01-03  Sat  7000        0

## 평일 중에 공휴일인 날이 있으면, 가격 바꿔주기

- price.loc[price["연도"] == "2009-07-02 00:00:00"]    # 183:2009행       # 공휴일 8000원
- price.loc[price["연도"] == "2013-02-13 00:00:00"]    # 2010:3331행      # 공휴일 9000원
- price.loc[price["연도"] == "2016-03-02 00:00:00"]    # 3332:4444행      # 공휴일 10000원
- price.loc[price["연도"] == "2018-04-10 00:00:00"]    # 4445:5213행      # 공휴일: 11000원
- price.loc[price["연도"] == "2020-10-25 00:00:00"]    # 5214:6142행      # 공휴일 : 12000원
- price.loc[price["연도"] == "2021-04-01 00:00:00"]    # 6143:6300행           # 공휴일 : 13000원
- price.loc[6301:, '가격']                             #                     공휴일 : 14000원

In [63]:
all_date.loc[183:2009,'가격'][all_date['holiday'] == 1] = 8000
all_date.loc[2010:3331,'가격'][all_date['holiday'] == 1] = 9000
all_date.loc[3332:4444,'가격'][all_date['holiday'] == 1] = 10000
all_date.loc[4445:5213,'가격' ][all_date['holiday'] == 1] = 11000
all_date.loc[5214:6142,'가격'][all_date['holiday'] == 1] = 12000
all_date.loc[6143:6300,'가격'][all_date['holiday'] == 1] = 13000
all_date.loc[6301:,'가격'][all_date['holiday'] == 1] = 14000

In [78]:
all_date.head(10)

date   요일    가격  holiday
0 2004-01-01  Thu  7000        1
1 2004-01-02  Fri  7000        0
2 2004-01-03  Sat  7000        0
3 2004-01-04  Sun  7000        0
4 2004-01-05  Mon  7000        0
5 2004-01-06  Tue  7000        0
6 2004-01-07  Wed  7000        0
7 2004-01-08  Thu  7000        0
8 2004-01-09  Fri  7000        0
9 2004-01-10  Sat  7000        0

## 공휴일 가격 까지 반영된 최종 가격표 checkpoint

In [65]:
all_date.to_csv("all_date.csv", encoding="utf-8")

## 기존의 audience1 데이터 프레임과 공휴일 가격 반영된 최종 가격표 까지 merge

In [79]:
audience1.info()
audience1.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7036 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   연도      7036 non-null   datetime64[ns]
 1   전체개봉편수  7036 non-null   int64         
 2   전체상영편수  7036 non-null   int64         
 3   전체매출액   7036 non-null   int64         
 4   전체관객수   7036 non-null   int64         
 5   요일      7036 non-null   object        
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 384.8+ KB


연도  전체개봉편수  전체상영편수      전체매출액   전체관객수   요일
0 2004-01-01       1       7  727005000  113837  Thu
1 2004-01-02       0       7  642276500  104357  Fri
2 2004-01-03       0       7  796519500  125234  Sat

In [77]:
all_date.info()
all_date.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7036 entries, 0 to 7035
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     7036 non-null   datetime64[ns]
 1   요일       7036 non-null   object        
 2   가격       7036 non-null   int64         
 3   holiday  7036 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 532.9+ KB


date   요일    가격  holiday
0 2004-01-01  Thu  7000        1
1 2004-01-02  Fri  7000        0
2 2004-01-03  Sat  7000        0

In [83]:
audience1.head()

연도  전체개봉편수  전체상영편수      전체매출액   전체관객수
0 2004-01-01       1       7  727005000  113837
1 2004-01-02       0       7  642276500  104357
2 2004-01-03       0       7  796519500  125234
3 2004-01-04       0       7  763271000  120203
4 2004-01-05       0       7  409269500   64180

In [88]:
audience1.rename(columns={"연도":"date"}, inplace=True)

In [89]:
Final = pd.merge(audience1, all_date, how='left')

In [91]:
Final.head()
Final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7036 entries, 0 to 7035
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     7036 non-null   datetime64[ns]
 1   전체개봉편수   7036 non-null   int64         
 2   전체상영편수   7036 non-null   int64         
 3   전체매출액    7036 non-null   int64         
 4   전체관객수    7036 non-null   int64         
 5   요일       7036 non-null   object        
 6   가격       7036 non-null   int64         
 7   holiday  7036 non-null   int64         
dtypes: datetime64[ns](1), int64(6), object(1)
memory usage: 494.7+ KB


In [93]:
Final.value_counts('date',ascending=False)       # 중복 행 확인 -> 없음


date
2004-01-01    1
2016-10-31    1
2016-11-11    1
2016-11-10    1
2016-11-09    1
             ..
2010-05-30    1
2010-05-29    1
2010-05-28    1
2010-05-27    1
2023-04-06    1
Length: 7036, dtype: int64

In [198]:
Final # 완성

연도  전체개봉편수  전체상영편수       전체매출액   전체관객수   요일     가격  holiday
0    2004-01-01       1       7   727005000  113837  Thu   7000        1
1    2004-01-02       0       7   642276500  104357  Fri   7000        0
2    2004-01-03       0       7   796519500  125234  Sat   7000        0
3    2004-01-04       0       7   763271000  120203  Sun   7000        0
4    2004-01-05       0       7   409269500   64180  Mon   7000        0
...         ...     ...     ...         ...     ...  ...    ...      ...
7031 2023-04-02       0      80  3423512979  336191  Sun  14000        0
7032 2023-04-03       0      87   802358725   81632  Mon  13000        0
7033 2023-04-04       2      91   815279367   82221  Tue  13000        0
7034 2023-04-05       6      94  1347838240  134732  Wed  13000        0
7035 2023-04-06       7     100  1058649412  107135  Thu  13000        0

[7036 rows x 8 columns]

# 전처리 끝

In [95]:
Final.to_csv("daily_audience.csv",encoding="utf-8")